In [1]:
import os ; os.environ["CUDA_VISIBLE_DEVICES"] = ""
import sys

In [2]:
from systemtools.hayj import *
from systemtools.location import *
from systemtools.basics import *
from systemtools.file import *
from systemtools.printer import *
from databasetools.mongo import *
from newstools.goodarticle.utils import *
from nlptools.preprocessing import *
from nlptools.news import parser as newsParser
from machinelearning.iterator import *
from twinews.utils import *
from twinews.models.ranking import *
from twinews.evaluation.utils import *

In [3]:
from nlptools.basics import *
from nltk.stem import WordNetLemmatizer 
from sklearn.decomposition import NMF, LatentDirichletAllocation
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from gensim.models import LdaMulticore
from gensim.test.utils import common_corpus, common_dictionary
from sklearn.metrics.pairwise import cosine_similarity, pairwise_distances
import gensim
from math import log2
from math import sqrt
from numpy import asarray

In [4]:
from twinews.yfnotebooks.dssm.graph import Graph
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
from twinews.yfnotebooks.dssm.load_data import hashIndex,load_hashed_data

Instructions for updating:
non-resource variables are not supported in the long term


In [5]:
isNotebook = '__file__' not in locals()
TEST = isNotebook

In [6]:
logger = Logger(tmpDir('logs') + "/dssm_yf.log") if isNotebook else Logger("dssm_yf-" + getHostname() + ".log")
tt = TicToc(logger=logger)
tt.tic()

--> tictoc starts...


-1

In [55]:
modelName = "DSSM_yf"

In [8]:
os.environ["KMP_DUPLICATE_LIB_OK"] = "TRUE"
os.environ["TF_CPP_MIN_LOG_LEVEL’"] = "3"

# config

In [11]:
config = \
{
    'splitVersion': 2,
    
    # for input data
    'maxUsers': 10 if TEST else None, # Sub-sampling
    'maxDocuments': 100 if TEST else 10000,
    'useExtraNews': 0 if TEST else None, # None = unlimited, 0 = no extra news
    'minDF': 1 / 500 if TEST else 1 / 2000, # Remove words that have a document frequency ratio lower than 1 / 500
    'maxDF': 300, # Remove top 300 voc elements
    'lowercase': False if TEST else True,
    'doLemmatization': False if TEST else False,
    #'useTFIDF': True,
    
   # for model    
    'seq_length' : 7427,
    'char_embedding_size': 100,
    'learning_rate': 0.0005,
    'keep_prob': 0.7,
    'vocab_size': 7901,
    'class_size': 2,
    'epochs': 10,
    'batch_size': 100,
    
    # for ouput
    'similarity': 'cosine',

    'historyRef': 0.3, # 1, 1.0, 0.5, 0.3, 3, 10
}

In [12]:
config['maxDocuments']

100

# get data

In [13]:
# Getting users and news
evalData = getEvalData(config['splitVersion'], maxExtraNews=0,
                       maxUsers=config['maxUsers'])
# (trainUsers, testUsers, trainNews, testNews, candidates, extraNews) = \
# (evalData['trainUsers'], evalData['testUsers'], evalData['trainNews'],
#  evalData['testNews'], evalData['candidates'], evalData['extraNews'])
bp(evalData.keys(), 5, logger)
#log(b(evalData['meta'], 5), logger)

--> tic: 18.02s | message: Eval data loaded


{ candidates, extraNews, meta, testNews, testUsers, trainNews, trainUsers }


--> toc total duration: 18.3s | message: Got Twinews evaluation data


In [14]:
(trainUsers, testUsers, trainNews, testNews, candidates, extraNews) = \
(evalData['trainUsers'], evalData['testUsers'], evalData['trainNews'],
 evalData['testNews'], evalData['candidates'], evalData['extraNews'])

In [15]:
extraNewsList = shuffle(list(extraNews), seed=0)
trainNewsList = shuffle(list(trainNews), seed=0)
testNewsList = shuffle(list(testNews), seed=0)

In [16]:
print(len(extraNewsList))
print(len(trainNewsList))
print(len(testNewsList))

0
299
9391


In [18]:
# train news as query
newsList = trainNewsList + testNewsList
log(str(len(newsList)) + " urls for newsList:\n" + b(newsList), logger=logger)

9690 urls for newsList:
[
  http://talkingpointsmemo.com/dc/roy-moores-pastor-ally-says-he-dated-teens-because-of-their-purity,
  http://www.dnaindia.com/delhi/report-delhi-smog-diesel-and-coal-particles-more-toxic-than-dust-claim,
  ...,
  http://bit.ly/2AHagU5,
  http://bit.ly/2DiRvJu
]


In [19]:
sentences = getNewsSentences(newsList, logger=logger)
bp(sentences, logger=logger)

twinews news (version 1.0) initialised.
  0% [                    ]
 10% [==                  ] (4m 30.18s left)
 20% [====                ] (3m 48.56s left)
 30% [======              ] (3m 17.703s left)
 40% [========            ] (2m 47.43s left)
 50% [==========          ] (2m 23.439s left)
 60% [============        ] (1m 57.053s left)
 70% [==============      ] (1m 28.967s left)
 80% [================    ] (1m 0.935s left)
 90% [==================  ] (30.81s left)
100% [====================] (total duration: 5m 9.629s, mean duration: 0.031s)
[
  [ [ A, prominent, ..., date, . ], [ Pastor, Flip, ..., girls, . ], ..., [ And, he, ..., old, . ], [ Benham, interview, ..., Watch, . ] ],
  [ [ Dust, may, ..., industries, . ], [ Studies, and, ..., air, . ], ..., [ ", In, ..., out, . ], [ It, estimates, ..., emission, " ] ],
  ...,
  [ [ The, Armenian, ..., groups, . ], [ The, draft, ..., Genocide, . ], ..., [ It, will, ..., violations, . ], [ The, draft, ..., January, __int_2__ ] ],
  [ [

In [23]:
for i in range(len(sentences)):
    sentences[i] = flattenLists(sentences[i])
docs = sentences
bp(docs, logger)

[ [ A, prominent, ..., Watch, . ], [ Dust, may, ..., emission, " ], ..., [ The, Armenian, ..., January, __int_2__ ], [ As, a, ..., space, . ] ]


In [24]:
if TEST:
    for i in pb(list(range(len(docs))), logger=logger, message="Lower casing"):
        for u in range(len(docs[i])):
            docs[i][u] = docs[i][u].lower()
bp(docs, logger)

Lower casing   0% [                    ]
Lower casing  10% [==                  ] (2.25s left)
Lower casing  20% [====                ] (1.839s left)
Lower casing  30% [======              ] (1.563s left)
Lower casing  40% [========            ] (1.319s left)
Lower casing  50% [==========          ] (1.09s left)
Lower casing  60% [============        ] (0.866s left)
Lower casing  70% [==============      ] (0.647s left)
Lower casing  80% [================    ] (0.432s left)
Lower casing  90% [==================  ] (0.216s left)
Lower casing 100% [====================] (total duration: 2.17s, mean duration: 0s)
[ [ a, prominent, ..., watch, . ], [ dust, may, ..., emission, " ], ..., [ the, armenian, ..., january, __int_2__ ], [ as, a, ..., space, . ] ]


In [25]:
if TEST:
    lemmatizer = WordNetLemmatizer()
    pbar = ProgressBar(len(docs), logger=logger, message="Lemmatization")
    for i in range(len(docs)):
        for u in range(len(docs[i])):
            docs[i][u] = lemmatizer.lemmatize(docs[i][u])
        pbar.tic()
bp(docs, logger)

Lemmatization   0% [                    ]
Lemmatization  10% [==                  ] (38.789s left)
Lemmatization  20% [====                ] (26.919s left)
Lemmatization  30% [======              ] (21.35s left)
Lemmatization  40% [========            ] (17.384s left)
Lemmatization  50% [==========          ] (13.99s left)
Lemmatization  60% [============        ] (10.906s left)
Lemmatization  70% [==============      ] (8.082s left)
Lemmatization  80% [================    ] (5.404s left)
Lemmatization  90% [==================  ] (2.71s left)
Lemmatization 100% [====================] (total duration: 27.33s, mean duration: 0.002s)
[ [ a, prominent, ..., watch, . ], [ dust, may, ..., emission, " ], ..., [ the, armenian, ..., january, __int_2__ ], [ a, a, ..., space, . ] ]


In [26]:
docs = filterCorpus(docs, minDF=1/2000, maxDF=300,
                    removeEmptyDocs=False, allowEmptyDocs=False, logger=logger)
for doc in docs: assert len(doc) > 0
bp(docs, logger)

Voc removed because of minDF (66696 elements):
{ 00s, 0640l, 0cean, 0day, 0f, 0x, 1, 10, 100, 1000th, ..., zzz, zzzs, 💗, 😂, 😃, 😉, 😍, 😒, 😢, 🙌 }
Voc removed because of maxDF (300 elements):
{ ", ', (, ), ,, -, ., :, ;, ?, ..., without, woman, work, working, world, would, year, yet, you, your }
70.36% of voc will be removed.
[ [ prominent, wing, ..., wing, watch ], [ dust, visible, ..., probably, emission ], ..., [ armenian, assembly, ..., sitting, january ], [ technology, analyst, ..., networking, space ] ]


In [27]:
def toStr(sentencesList):
    for i in range(len(sentencesList)):
        sentencesList[i] = ' '.join(sentencesList[i])
    return sentencesList

corpus = toStr(docs)
for corp in corpus: assert len(corp) > 0
bp(corpus,logger)

[
  prominent wing preacher appeared alongside senate candidate roy moore campaign rally ago moore dated,
  dust visible marker delhi air pollution sheer abundance actually masking bigger killer emission vehi,
  ...,
  armenian assembly standing committee foreign affair consider draft statement condemning genocide ter,
  technology analyst asked living simply stated paid provide opinion sector cover dream ask friend eve
]


In [28]:
urlDocs = dict()
for i in range(len(newsList)):
    urlDocs[newsList[i]] = corpus[i]
bp(urlDocs, 2, logger)

{
  http://247wallst.com/special-report/2017/12/08/americas-25-dying-industries-2: estimated __float_3__ united __float_1__ ago economy whole grew healthy pace workforce growth unifor,
  http://247wallst.com/special-report/2018/01/11/25-best-fitness-tech-gadgets/: looking resolution extra pound holiday surprisingly looking invest nascent field wearable tech fitne,
  http://a.msn.com/00/en-us/AAuyJLc?ocid=st: wal mart nyse decided raise minimum wage pay worker hour following footstep target nyse september de,
  http://a.msn.com/00/en-us/BBHoy0Q?ocid=st: richest earth became trillion richer gain stock market shrugged economic social political division r,
  http://a.msn.com/01/en-ie/BBHzIAS?ocid=st: brought stunning rare wolf moon light sky monday evening lunar cycle cycle align astronomical treat ,
  ...,
  https://www.zerohedge.com/news/2018-01-08/credit-card-debt-hits-all-time-high-consumers-unleash-historic-shopping-spree: reason behind rebound economy explained word charge confirmati

In [57]:
a = []
a.append(urlDocs['http://247wallst.com/special-report/2017/12/08/americas-25-dying-industries-2'])
a

['estimated __float_3__ united __float_1__ ago economy whole grew healthy pace workforce growth uniform handful industry doubled size shed half workforce nearly industry economy subject powerful unpredictable force free market globalization technological advancement fundamentally changed economic landscape worse industry seemed unassailable appear hanging thread wall st. reviewed annual employment data bureau labor statistic identify fastest dying industry industry list shed worker lower nearly fastest dying industry']

In [29]:
urlHasedDocs = dict()
for key in urlDocs.keys():
    temp = []
    temp.append(urlDocs[key])
    urlHasedDocs[key], _  = hashIndex(temp, ' ')
bp(urlHasedDocs)

{
  http://247wallst.com/special-report/2017/12/08/americas-25-dying-industries-2: (0, 54) 1.0
(0, 89) 1.0
(0, 93) 2.0
(0, 98) 1.0
(0, 117) 1.0
(0, 124) 2.0
(0, 128) 1.0
(0, 131) 1.0
...,
  http://247wallst.com/special-report/2018/01/11/25-best-fitness-tech-gadgets/: (0, 1) 1.0
(0, 23) 1.0
(0, 32) 5.0
(0, 47) 1.0
(0, 52) 1.0
(0, 54) 1.0
(0, 73) 9.0
(0, 83) 1.0
(0, 8...,
  http://a.msn.com/00/en-us/AAuyJLc?ocid=st: (0, 14) 2.0
(0, 61) 14.0
(0, 62) 1.0
(0, 63) 1.0
(0, 78) 1.0
(0, 93) 1.0
(0, 124) 1.0
(0, 128) 1.0
(...,
  http://a.msn.com/00/en-us/BBHoy0Q?ocid=st: (0, 22) 2.0
(0, 23) 1.0
(0, 24) 1.0
(0, 32) 12.0
(0, 54) 2.0
(0, 61) 2.0
(0, 62) 2.0
(0, 63) 2.0
(0,...,
  http://a.msn.com/01/en-ie/BBHzIAS?ocid=st: (0, 1) 1.0
(0, 29) 1.0
(0, 32) 1.0
(0, 52) 1.0
(0, 59) 3.0
(0, 61) 1.0
(0, 67) 1.0
(0, 78) 1.0
(0, 8...,
  ...,
  https://www.zerohedge.com/news/2018-01-08/credit-card-debt-hits-all-time-high-consumers-unleash-historic-shopping-spree: (0, 32) 2.0
(0, 38) 1.0
(0, 54) 1.0
(0, 59) 1.0

In [30]:
bp(trainUsers.keys())

{ 1198644368, 21379050, ..., 839264223741956096, 97081361 }


# news classification

In [32]:
# for userId in trainUsers.keys():
query = np.zeros((1,7427))
for url in evalData['trainUsers']['1198644368']:
    query += urlHasedDocs[url].toarray() # sum the history urls of users
bp(query,2)

[[ 0. 25.  0. ...  0. 21.  0.]]


In [48]:
# p, h, y = load_hashed_data('/home/yuting/PycharmProjects/data/dssm_test_dev_sub.csv', data_size=None)
p = query
h = urlHasedDocs['http://a.msn.com/01/en-ie/BBHzIAS?ocid=st'].toarray()
y = 0

def computeSimilarity(p,h,y):
    
    tf.reset_default_graph() 
    model = Graph()
    saver = tf.train.Saver()

    with tf.Session()as sess:
        sess.run(tf.global_variables_initializer())
        saver.restore(sess, '/home/yuting/PycharmProjects/Twinews/twinews/yfnotebooks/dssm/output/dssm_9.ckpt')
        loss, acc, logit = sess.run([model.loss, model.acc, model.logits],
                             feed_dict={model.p: p,
                                        model.h: h,
                                        model.y: y,
                                        model.keep_prob: 1})

        #print('loss: ', loss, ' acc:', acc)
        return logit
logit = computeSimilarity(p,h,y)

INFO:tensorflow:Restoring parameters from /home/yuting/PycharmProjects/Twinews/twinews/yfnotebooks/dssm/output/dssm_9.ckpt


In [50]:
logit[0][0]

0.66722286

In [51]:
# rankings of the candidates
rankings = dict()
for usr in trainUsers.keys():
    query = np.zeros((1,7427))
    for url in evalData['trainUsers'][usr]:
        query += urlHasedDocs[url].toarray()
    usrRankings = []
    for candidates in evalData['candidates'][usr]:
        candidates = list(candidates)
        scoresCandidates = []
        for i in range(len(candidates)):
            candidateHashed = urlHasedDocs[candidates[i]].toarray()
            logit = computeSimilarity(query, candidateHashed, 0)
            score = logit[0][0]
            # score = 1
            scoresCandidates.append((candidates[i], score))
        ranking = sortBy(scoresCandidates, index=1, desc=True)
        ranking = [e[0] for e in ranking]
        usrRankings.append(ranking)
    rankings[usr] = usrRankings

INFO:tensorflow:Restoring parameters from /home/yuting/PycharmProjects/Twinews/twinews/yfnotebooks/dssm/output/dssm_9.ckpt
INFO:tensorflow:Restoring parameters from /home/yuting/PycharmProjects/Twinews/twinews/yfnotebooks/dssm/output/dssm_9.ckpt
INFO:tensorflow:Restoring parameters from /home/yuting/PycharmProjects/Twinews/twinews/yfnotebooks/dssm/output/dssm_9.ckpt
INFO:tensorflow:Restoring parameters from /home/yuting/PycharmProjects/Twinews/twinews/yfnotebooks/dssm/output/dssm_9.ckpt
INFO:tensorflow:Restoring parameters from /home/yuting/PycharmProjects/Twinews/twinews/yfnotebooks/dssm/output/dssm_9.ckpt
INFO:tensorflow:Restoring parameters from /home/yuting/PycharmProjects/Twinews/twinews/yfnotebooks/dssm/output/dssm_9.ckpt
INFO:tensorflow:Restoring parameters from /home/yuting/PycharmProjects/Twinews/twinews/yfnotebooks/dssm/output/dssm_9.ckpt
INFO:tensorflow:Restoring parameters from /home/yuting/PycharmProjects/Twinews/twinews/yfnotebooks/dssm/output/dssm_9.ckpt
INFO:tensorflow:

INFO:tensorflow:Restoring parameters from /home/yuting/PycharmProjects/Twinews/twinews/yfnotebooks/dssm/output/dssm_9.ckpt
INFO:tensorflow:Restoring parameters from /home/yuting/PycharmProjects/Twinews/twinews/yfnotebooks/dssm/output/dssm_9.ckpt
INFO:tensorflow:Restoring parameters from /home/yuting/PycharmProjects/Twinews/twinews/yfnotebooks/dssm/output/dssm_9.ckpt
INFO:tensorflow:Restoring parameters from /home/yuting/PycharmProjects/Twinews/twinews/yfnotebooks/dssm/output/dssm_9.ckpt
INFO:tensorflow:Restoring parameters from /home/yuting/PycharmProjects/Twinews/twinews/yfnotebooks/dssm/output/dssm_9.ckpt
INFO:tensorflow:Restoring parameters from /home/yuting/PycharmProjects/Twinews/twinews/yfnotebooks/dssm/output/dssm_9.ckpt
INFO:tensorflow:Restoring parameters from /home/yuting/PycharmProjects/Twinews/twinews/yfnotebooks/dssm/output/dssm_9.ckpt
INFO:tensorflow:Restoring parameters from /home/yuting/PycharmProjects/Twinews/twinews/yfnotebooks/dssm/output/dssm_9.ckpt
INFO:tensorflow:

INFO:tensorflow:Restoring parameters from /home/yuting/PycharmProjects/Twinews/twinews/yfnotebooks/dssm/output/dssm_9.ckpt
INFO:tensorflow:Restoring parameters from /home/yuting/PycharmProjects/Twinews/twinews/yfnotebooks/dssm/output/dssm_9.ckpt
INFO:tensorflow:Restoring parameters from /home/yuting/PycharmProjects/Twinews/twinews/yfnotebooks/dssm/output/dssm_9.ckpt
INFO:tensorflow:Restoring parameters from /home/yuting/PycharmProjects/Twinews/twinews/yfnotebooks/dssm/output/dssm_9.ckpt
INFO:tensorflow:Restoring parameters from /home/yuting/PycharmProjects/Twinews/twinews/yfnotebooks/dssm/output/dssm_9.ckpt
INFO:tensorflow:Restoring parameters from /home/yuting/PycharmProjects/Twinews/twinews/yfnotebooks/dssm/output/dssm_9.ckpt
INFO:tensorflow:Restoring parameters from /home/yuting/PycharmProjects/Twinews/twinews/yfnotebooks/dssm/output/dssm_9.ckpt
INFO:tensorflow:Restoring parameters from /home/yuting/PycharmProjects/Twinews/twinews/yfnotebooks/dssm/output/dssm_9.ckpt
INFO:tensorflow:

INFO:tensorflow:Restoring parameters from /home/yuting/PycharmProjects/Twinews/twinews/yfnotebooks/dssm/output/dssm_9.ckpt
INFO:tensorflow:Restoring parameters from /home/yuting/PycharmProjects/Twinews/twinews/yfnotebooks/dssm/output/dssm_9.ckpt
INFO:tensorflow:Restoring parameters from /home/yuting/PycharmProjects/Twinews/twinews/yfnotebooks/dssm/output/dssm_9.ckpt
INFO:tensorflow:Restoring parameters from /home/yuting/PycharmProjects/Twinews/twinews/yfnotebooks/dssm/output/dssm_9.ckpt
INFO:tensorflow:Restoring parameters from /home/yuting/PycharmProjects/Twinews/twinews/yfnotebooks/dssm/output/dssm_9.ckpt
INFO:tensorflow:Restoring parameters from /home/yuting/PycharmProjects/Twinews/twinews/yfnotebooks/dssm/output/dssm_9.ckpt
INFO:tensorflow:Restoring parameters from /home/yuting/PycharmProjects/Twinews/twinews/yfnotebooks/dssm/output/dssm_9.ckpt
INFO:tensorflow:Restoring parameters from /home/yuting/PycharmProjects/Twinews/twinews/yfnotebooks/dssm/output/dssm_9.ckpt
INFO:tensorflow:

INFO:tensorflow:Restoring parameters from /home/yuting/PycharmProjects/Twinews/twinews/yfnotebooks/dssm/output/dssm_9.ckpt
INFO:tensorflow:Restoring parameters from /home/yuting/PycharmProjects/Twinews/twinews/yfnotebooks/dssm/output/dssm_9.ckpt
INFO:tensorflow:Restoring parameters from /home/yuting/PycharmProjects/Twinews/twinews/yfnotebooks/dssm/output/dssm_9.ckpt
INFO:tensorflow:Restoring parameters from /home/yuting/PycharmProjects/Twinews/twinews/yfnotebooks/dssm/output/dssm_9.ckpt
INFO:tensorflow:Restoring parameters from /home/yuting/PycharmProjects/Twinews/twinews/yfnotebooks/dssm/output/dssm_9.ckpt
INFO:tensorflow:Restoring parameters from /home/yuting/PycharmProjects/Twinews/twinews/yfnotebooks/dssm/output/dssm_9.ckpt
INFO:tensorflow:Restoring parameters from /home/yuting/PycharmProjects/Twinews/twinews/yfnotebooks/dssm/output/dssm_9.ckpt
INFO:tensorflow:Restoring parameters from /home/yuting/PycharmProjects/Twinews/twinews/yfnotebooks/dssm/output/dssm_9.ckpt
INFO:tensorflow:

...

INFO:tensorflow:Restoring parameters from /home/yuting/PycharmProjects/Twinews/twinews/yfnotebooks/dssm/output/dssm_9.ckpt
INFO:tensorflow:Restoring parameters from /home/yuting/PycharmProjects/Twinews/twinews/yfnotebooks/dssm/output/dssm_9.ckpt
INFO:tensorflow:Restoring parameters from /home/yuting/PycharmProjects/Twinews/twinews/yfnotebooks/dssm/output/dssm_9.ckpt
INFO:tensorflow:Restoring parameters from /home/yuting/PycharmProjects/Twinews/twinews/yfnotebooks/dssm/output/dssm_9.ckpt
INFO:tensorflow:Restoring parameters from /home/yuting/PycharmProjects/Twinews/twinews/yfnotebooks/dssm/output/dssm_9.ckpt
INFO:tensorflow:Restoring parameters from /home/yuting/PycharmProjects/Twinews/twinews/yfnotebooks/dssm/output/dssm_9.ckpt
INFO:tensorflow:Restoring parameters from /home/yuting/PycharmProjects/Twinews/twinews/yfnotebooks/dssm/output/dssm_9.ckpt
INFO:tensorflow:Restoring parameters from /home/yuting/PycharmProjects/Twinews/twinews/yfnotebooks/dssm/output/dssm_9.ckpt
INFO:tensorflow:

INFO:tensorflow:Restoring parameters from /home/yuting/PycharmProjects/Twinews/twinews/yfnotebooks/dssm/output/dssm_9.ckpt
INFO:tensorflow:Restoring parameters from /home/yuting/PycharmProjects/Twinews/twinews/yfnotebooks/dssm/output/dssm_9.ckpt
INFO:tensorflow:Restoring parameters from /home/yuting/PycharmProjects/Twinews/twinews/yfnotebooks/dssm/output/dssm_9.ckpt
INFO:tensorflow:Restoring parameters from /home/yuting/PycharmProjects/Twinews/twinews/yfnotebooks/dssm/output/dssm_9.ckpt
INFO:tensorflow:Restoring parameters from /home/yuting/PycharmProjects/Twinews/twinews/yfnotebooks/dssm/output/dssm_9.ckpt
INFO:tensorflow:Restoring parameters from /home/yuting/PycharmProjects/Twinews/twinews/yfnotebooks/dssm/output/dssm_9.ckpt
INFO:tensorflow:Restoring parameters from /home/yuting/PycharmProjects/Twinews/twinews/yfnotebooks/dssm/output/dssm_9.ckpt
INFO:tensorflow:Restoring parameters from /home/yuting/PycharmProjects/Twinews/twinews/yfnotebooks/dssm/output/dssm_9.ckpt
INFO:tensorflow:

INFO:tensorflow:Restoring parameters from /home/yuting/PycharmProjects/Twinews/twinews/yfnotebooks/dssm/output/dssm_9.ckpt
INFO:tensorflow:Restoring parameters from /home/yuting/PycharmProjects/Twinews/twinews/yfnotebooks/dssm/output/dssm_9.ckpt
INFO:tensorflow:Restoring parameters from /home/yuting/PycharmProjects/Twinews/twinews/yfnotebooks/dssm/output/dssm_9.ckpt
INFO:tensorflow:Restoring parameters from /home/yuting/PycharmProjects/Twinews/twinews/yfnotebooks/dssm/output/dssm_9.ckpt
INFO:tensorflow:Restoring parameters from /home/yuting/PycharmProjects/Twinews/twinews/yfnotebooks/dssm/output/dssm_9.ckpt
INFO:tensorflow:Restoring parameters from /home/yuting/PycharmProjects/Twinews/twinews/yfnotebooks/dssm/output/dssm_9.ckpt
INFO:tensorflow:Restoring parameters from /home/yuting/PycharmProjects/Twinews/twinews/yfnotebooks/dssm/output/dssm_9.ckpt
INFO:tensorflow:Restoring parameters from /home/yuting/PycharmProjects/Twinews/twinews/yfnotebooks/dssm/output/dssm_9.ckpt
INFO:tensorflow:

INFO:tensorflow:Restoring parameters from /home/yuting/PycharmProjects/Twinews/twinews/yfnotebooks/dssm/output/dssm_9.ckpt
INFO:tensorflow:Restoring parameters from /home/yuting/PycharmProjects/Twinews/twinews/yfnotebooks/dssm/output/dssm_9.ckpt
INFO:tensorflow:Restoring parameters from /home/yuting/PycharmProjects/Twinews/twinews/yfnotebooks/dssm/output/dssm_9.ckpt
INFO:tensorflow:Restoring parameters from /home/yuting/PycharmProjects/Twinews/twinews/yfnotebooks/dssm/output/dssm_9.ckpt
INFO:tensorflow:Restoring parameters from /home/yuting/PycharmProjects/Twinews/twinews/yfnotebooks/dssm/output/dssm_9.ckpt
INFO:tensorflow:Restoring parameters from /home/yuting/PycharmProjects/Twinews/twinews/yfnotebooks/dssm/output/dssm_9.ckpt
INFO:tensorflow:Restoring parameters from /home/yuting/PycharmProjects/Twinews/twinews/yfnotebooks/dssm/output/dssm_9.ckpt
INFO:tensorflow:Restoring parameters from /home/yuting/PycharmProjects/Twinews/twinews/yfnotebooks/dssm/output/dssm_9.ckpt
INFO:tensorflow:

INFO:tensorflow:Restoring parameters from /home/yuting/PycharmProjects/Twinews/twinews/yfnotebooks/dssm/output/dssm_9.ckpt
INFO:tensorflow:Restoring parameters from /home/yuting/PycharmProjects/Twinews/twinews/yfnotebooks/dssm/output/dssm_9.ckpt
INFO:tensorflow:Restoring parameters from /home/yuting/PycharmProjects/Twinews/twinews/yfnotebooks/dssm/output/dssm_9.ckpt
INFO:tensorflow:Restoring parameters from /home/yuting/PycharmProjects/Twinews/twinews/yfnotebooks/dssm/output/dssm_9.ckpt
INFO:tensorflow:Restoring parameters from /home/yuting/PycharmProjects/Twinews/twinews/yfnotebooks/dssm/output/dssm_9.ckpt
INFO:tensorflow:Restoring parameters from /home/yuting/PycharmProjects/Twinews/twinews/yfnotebooks/dssm/output/dssm_9.ckpt
INFO:tensorflow:Restoring parameters from /home/yuting/PycharmProjects/Twinews/twinews/yfnotebooks/dssm/output/dssm_9.ckpt
INFO:tensorflow:Restoring parameters from /home/yuting/PycharmProjects/Twinews/twinews/yfnotebooks/dssm/output/dssm_9.ckpt
INFO:tensorflow:

In [52]:
bp(rankings,2)

{
  1198644368: 
  [
    [
      http://tinyurl.com/ycvt6rp5,
      https://www.bizjournals.com/nashville/news/2018/01/11/who-would-play-nashvilles-ceos-in-the-movies-c,
      ...,
      https://www.washingtonpost.com/lifestyle/magazine/dave-barrys-year-in-review-russia-mania-covfefe-an,
      https://www.hrw.org/report/2007/09/11/no-easy-answers/sex-offender-laws-us
    ]
  ],
  21379050: 
  [
    [
      https://www.wsj.com/articles/the-great-rules-rollback-1514237372,
      http://www.telegraph.co.uk/news/2017/12/31/ministers-use-artificial-intelligence-target-mass-benefit,
      ...,
      http://time.com/69316/basic/,
      http://ow.ly/YTHL30hlDxp
    ]
  ],
  2617447752: 
  [
    [
      https://www.csindy.com/SideDish/archives/2017/12/29/mccabes-to-close-become-streetcar-520,
      https://pamboyd.wordpress.com/2018/01/02/while-thinking-over-2017/,
      ...,
      https://www.theatlantic.com/magazine/archive/1901/01/the-reconstruction-of-the-southern-states/52003,
      http:/

In [57]:
addRanking(modelName, rankings, config, logger=logger)

In [56]:
modelName

'DSSM_yf'

In [134]:
bp(evalData['candidates'],2)

{
  1198644368: 
  [
    {
      http://a.msn.com/00/en-us/AAuyJLc?ocid=st,
      http://a.msn.com/00/en-us/BBHoy0Q?ocid=st,
      ...,
      https://www.wxyz.com/news/opinion/wxyz-editorial-tv-7-newsmaker-of-the-year-gerard-gerry-anderson-en,
      https://www.zerohedge.com/news/2018-01-08/grand-jury-empaneled-10-million-fraud-probe-involving-jane
    }
  ],
  21379050: 
  [
    {
      http://a.msn.com/01/en-us/AAuDxQJ?ocid=st,
      http://a.msn.com/01/en-us/BBIaq0E?ocid=st,
      ...,
      https://www.yahoo.com/news/ancient-egypt-mysterious-2-600-133623030.html?.tsrc=fauxdal,
      https://www.yourtango.com/2017309426/10-tiny-things-men-who-will-commit-marry-you-do-differently-rel
    }
  ],
  2617447752: 
  [
    {
      http://a.msn.com/06/en-us/BBH7EiO?ocid=st,
      http://abcn.ws/2CbqxlW,
      ...,
      https://www.zerohedge.com/news/2018-01-03/children-forced-freeze-baltimore-schools-open-no-heat,
      https://yaledailynews.com/blog/2018/01/06/yale-psychiatrist-met-with-c

In [22]:
len(candidates['1198644368'])

1

In [53]:
checkRankings(rankings,evalData['candidates'],maxUsers=None)

# Training data

In [ ]:
# Getting users and news
evalData = getEvalData(config['splitVersion'], maxExtraNews=config['maxDocuments'],
                       maxUsers=config['maxUsers'], logger=logger)
(trainUsers, testUsers, trainNews, testNews, candidates, extraNews) = \
(evalData['trainUsers'], evalData['testUsers'], evalData['trainNews'],
 evalData['testNews'], evalData['candidates'], evalData['extraNews'])
bp(evalData.keys(), 5, logger)
log(b(evalData['meta'], 5), logger)